# Dataset 1

In [4]:
from libs.foxutils.utils.core_utils import settings, datasets_dir
import pandas as pd 
import numpy as np 
from os.path import join as pathjoin 

dataset_name = settings["DATASET"]["dataset"]
project_name = settings["RUN"]["project_name"]

In [41]:
print(f"Run project {project_name} for data {dataset_name}")
target_name = "slice Type"

def preprocess(filename = "train.csv"):
    datafile = pathjoin(datasets_dir, project_name, dataset_name, filename)
    df = pd.read_csv(datafile)
    if not (target_name in df.columns):
        target_col = None
    else:
        target_col = df[target_name]
        df.drop(columns=[target_name], inplace=True)
    return df, target_col 

df, target_col = preprocess("train.csv")

num_cols = len(df.columns)
num_rows = len(df)
num_classes = len(np.unique(target_col))

print(f"Num cols: {num_cols}. Num rows: {num_rows}. Num classes: {num_classes}")
print(f"Name cols: {df.columns}")
print(f"Target col: {target_name}")
df.head()


Run project explor for data 5g-network-slicing-analysis
Num cols: 16. Num rows: 31583. Num classes: 3
Name cols: Index(['LTE/5g Category', 'Time', 'Packet Loss Rate', 'Packet delay', 'IoT',
       'LTE/5G', 'GBR', 'Non-GBR', 'AR/VR/Gaming', 'Healthcare',
       'Industry 4.0', 'IoT Devices', 'Public Safety', 'Smart City & Home',
       'Smart Transportation', 'Smartphone'],
      dtype='object')
Target col: slice Type


,LTE/5g Category,Time,Packet Loss Rate,Packet delay,IoT,LTE/5G,GBR,Non-GBR,AR/VR/Gaming,Healthcare,Industry 4.0,IoT Devices,Public Safety,Smart City & Home,Smart Transportation,Smartphone
0,14,0,0.000001,10,1,0,0,1,0,0,0,0,1,0,0,0
1,18,20,0.001000,100,0,1,1,0,1,0,0,0,0,0,0,0
2,17,14,0.000001,300,0,1,0,1,0,0,0,0,0,0,0,1
3,3,17,0.010000,100,0,1,0,1,0,0,0,0,0,0,0,1
4,9,4,0.010000,50,1,0,0,1,0,0,0,0,0,1,0,0


In [30]:
### Sandardization of data ###
### We does not standardize the Target variable for classification
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
 
# Storing the fit object for later reference
scaler.fit(df)

X_train = scaler.transform(df)
y_train = pd.get_dummies(target_col)


In [31]:
import tensorflow as tf
num_inputs = len(df.columns)
BATCH_SIZE = 32
EPOCHS = 10

classifier = tf.keras.Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(tf.keras.layers.Dense(units=10, input_dim=num_inputs, kernel_initializer='uniform', activation='relu'))
 
#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(tf.keras.layers.Dense(units=6, kernel_initializer='uniform', activation='relu'))
 
# Defining the Output layer
# sigmoid means sigmoid activation function
# for Multiclass classification the activation ='softmax'
# And output_dim will be equal to the number of factor levels
classifier.add(tf.keras.layers.Dense(units=num_classes, kernel_initializer='uniform', activation='softmax'))
 
# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(X_train, y_train, batch_size=BATCH_SIZE , epochs=EPOCHS, verbose=1)
 

Epoch 1/10
987/987 [==============================] - 3s 2ms/step - loss: 0.3051 - accuracy: 0.9211
Epoch 2/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0677 - accuracy: 1.0000
Epoch 3/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0289 - accuracy: 1.0000
Epoch 4/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 5/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 6/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 7/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 8/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 9/10
987/987 [==============================] - 2s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 10/10
987/987 [==============================] - 2s 2ms/step - loss: 6.5426e-04 - accuracy: 1.

In [48]:
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 10)                170       
                                                                 
 dense_16 (Dense)            (None, 6)                 66        
                                                                 
 dense_17 (Dense)            (None, 3)                 21        
                                                                 
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Predictions on testing data
df_test, target_col_test = preprocess("test.csv")

X_test = scaler.transform(df_test)

preds = classifier.predict(X_test)

987/987 [==============================] - 1s 988us/step


In [46]:
preds

array([[9.9999964e-01, 1.9662869e-08, 3.4845888e-07],
       [1.2301930e-03, 9.5011014e-04, 9.9781966e-01],
       [5.4794606e-07, 9.9999285e-01, 6.5192103e-06],
       ...,
       [1.2301930e-03, 9.5011014e-04, 9.9781966e-01],
       [9.9999905e-01, 4.3561009e-08, 9.2821159e-07],
       [1.2301930e-03, 9.5011014e-04, 9.9781966e-01]], dtype=float32)

# Dataset 2

In [52]:
from libs.foxutils.utils.core_utils import settings, datasets_dir
import pandas as pd 
import numpy as np 
from os.path import join as pathjoin 

dataset_name = "networkslicing"
project_name = settings["RUN"]["project_name"]


In [58]:
print(f"Run project {project_name} for data {dataset_name}")
target_name = "slice Type"

def preprocess(filename = "train.csv"):
    datafile = pathjoin(datasets_dir, project_name, dataset_name, filename)
    df = pd.read_csv(datafile, index_col=0)
    if not (target_name in df.columns):
        target_col = None
    else:
        target_col = df[target_name]
        df.drop(columns=[target_name], inplace=True)
    
    df["Use Case"]=df["Use Case"].astype('category').cat.codes
    df["Technology Supported"]=df["Technology Supported"].astype('category').cat.codes
    df["Day"]=df["Day"].astype('category').cat.codes
    df["GBR"]=df["GBR"].astype('category').cat.codes

    return df, target_col 

df, target_col = preprocess("train.csv")

num_cols = len(df.columns)
num_rows = len(df)
num_classes = len(np.unique(target_col))

print(f"Num cols: {num_cols}. Num rows: {num_rows}. Num classes: {num_classes}")
print(f"Name cols: {df.columns}")
print(f"Target col: {target_name}")
df.head()


Run project explor for data networkslicing
Num cols: 8. Num rows: 63167. Num classes: 3
Name cols: Index(['Use Case', 'LTE/5g Category', 'Technology Supported', 'Day', 'Time',
       'GBR', 'Packet Loss Rate', 'Packet delay'],
      dtype='object')
Target col: slice Type


,Use Case,LTE/5g Category,Technology Supported,Day,Time,GBR,Packet Loss Rate,Packet delay
1,7,1,1,3,1,1,0.01,100
2,7,1,1,3,2,1,0.01,100
3,7,1,1,3,3,1,0.01,100
4,7,1,1,3,4,1,0.01,100
5,7,1,1,3,5,1,0.01,100


In [59]:
### Sandardization of data ###
### We does not standardize the Target variable for classification
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
 
# Storing the fit object for later reference
scaler.fit(df)

X_train = scaler.transform(df)
y_train = pd.get_dummies(target_col)

In [60]:
import tensorflow as tf
num_inputs = len(df.columns)
BATCH_SIZE = 32
EPOCHS = 10

classifier = tf.keras.Sequential()
# Defining the Input layer and FIRST hidden layer,both are same!
# relu means Rectifier linear unit function
classifier.add(tf.keras.layers.Dense(units=10, input_dim=num_inputs, kernel_initializer='uniform', activation='relu'))
 
#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(tf.keras.layers.Dense(units=6, kernel_initializer='uniform', activation='relu'))
 
# Defining the Output layer
# sigmoid means sigmoid activation function
# for Multiclass classification the activation ='softmax'
# And output_dim will be equal to the number of factor levels
classifier.add(tf.keras.layers.Dense(units=num_classes, kernel_initializer='uniform', activation='softmax'))
 
# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(X_train, y_train, batch_size=BATCH_SIZE , epochs=EPOCHS, verbose=1)

Epoch 1/10
1974/1974 [==============================] - 4s 2ms/step - loss: 0.1463 - accuracy: 0.9465
Epoch 2/10
1974/1974 [==============================] - 4s 2ms/step - loss: 2.1064e-04 - accuracy: 1.0000
Epoch 3/10
1974/1974 [==============================] - 4s 2ms/step - loss: 3.6386e-05 - accuracy: 1.0000
Epoch 4/10
1974/1974 [==============================] - 4s 2ms/step - loss: 9.7791e-06 - accuracy: 1.0000
Epoch 5/10
1974/1974 [==============================] - 4s 2ms/step - loss: 2.9920e-06 - accuracy: 1.0000
Epoch 6/10
1974/1974 [==============================] - 4s 2ms/step - loss: 9.7699e-07 - accuracy: 1.0000
Epoch 7/10
1974/1974 [==============================] - 4s 2ms/step - loss: 3.2251e-07 - accuracy: 1.0000
Epoch 8/10
1974/1974 [==============================] - 4s 2ms/step - loss: 8.6638e-08 - accuracy: 1.0000
Epoch 9/10
1974/1974 [==============================] - 4s 2ms/step - loss: 2.1418e-08 - accuracy: 1.0000
Epoch 10/10
1974/1974 [===========================

In [61]:
classifier.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 10)                90        
                                                                 
 dense_19 (Dense)            (None, 6)                 66        
                                                                 
 dense_20 (Dense)            (None, 3)                 21        
                                                                 
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


# Dataset 3

In [62]:
from libs.foxutils.utils.core_utils import settings, datasets_dir
import pandas as pd 
import numpy as np 
from os.path import join as pathjoin 

dataset_name = "small_cell_nodes"
project_name = settings["RUN"]["project_name"]

In [67]:
print(f"Run project {project_name} for data {dataset_name}")
target_name = "slice Type"

def preprocess(filename = "train.csv"):
    datafile = pathjoin(datasets_dir, project_name, dataset_name, filename)
    df = pd.read_csv(datafile, index_col=None)
    if not (target_name in df.columns):
        target_col = None
    else:
        target_col = df[target_name]
        df.drop(columns=[target_name], inplace=True)
    
    #df["Use Case"]=df["Use Case"].astype('category').cat.codes

    return df, target_col 

df, target_col = preprocess("train.csv")

num_cols = len(df.columns)
num_rows = len(df)
num_classes = len(np.unique(target_col))

print(f"Num cols: {num_cols}. Num rows: {num_rows}. Num classes: {num_classes}")
print(f"Name cols: {df.columns}")
print(f"Target col: {target_name}")
df.head()

Run project explor for data small_cell_nodes
Num cols: 22. Num rows: 2391. Num classes: 1
Name cols: Index(['OWNERNAME', 'APPLICANTCOMPANYNAME', 'PERMITMASTERNUMBER',
       'PERMITREFNUMBER', 'NODEGROUP', 'NODEID', 'NODEADDRESS', 'NODE_N',
       'NODE_E', 'NODETYPE', 'STATUS', 'POLEMATERIAL', 'POLEMANUFACTURER',
       'POLEMODEL', 'POLEHEIGHT_FT', 'POLECOLOR', 'RFEQUIPMENT', 'INTENDEDUSE',
       'ASSETOWNER', 'SITEID', 'FIXEDASSETID', 'COUNCIL_DISTRICT'],
      dtype='object')
Target col: slice Type


,OWNERNAME,APPLICANTCOMPANYNAME,PERMITMASTERNUMBER,PERMITREFNUMBER,NODEGROUP,NODEID,NODEADDRESS,NODE_N,NODE_E,NODETYPE,...,POLEMANUFACTURER,POLEMODEL,POLEHEIGHT_FT,POLECOLOR,RFEQUIPMENT,INTENDEDUSE,ASSETOWNER,SITEID,FIXEDASSETID,COUNCIL_DISTRICT
0,Mobilitie,Mobilitie,NaN,NaN,2018 Group 1A,DN90XS379E,1692 Glenarm PL,39.744554,-104.989407,"Type 3A, Combo Pole w/Cantenna",...,CompTek,1002661,30.0,Federal Green (RAL 6012),CCI SCA-DR1-65F6-E-H2S-K(Panel Ant),Single Carrier,,NaN,NaN,9
1,Mobilitie,Mobilitie,NaN,NaN,2018 Group 3\r\n,DN90XS381B\r\n,1652 N. Lincoln St.\r\n,39.742565,-104.985513,"Type 1A, Utility Pole Mounted",...,Other,TBD by Xcel\r\n,34.0,,Alpha AW3477-S (Omni B41),Single Carrier,,NaN,NaN,10
2,Mobilitie,Mobilitie,NaN,NaN,2018 Group 1A\r\n,DN90XSG53C\r\n,1688 Welton St.\r\n,39.745220,-104.990277,"Type 3A, Combo Pole w/Cantenna",...,CompTek,1002661,30.0,Federal Green (RAL 6012),CCI SCA-DR1-65F6-E-H2S-K(Panel Ant),Single Carrier,,NaN,NaN,9
3,Mobilitie,Mobilitie,NaN,NaN,2018 Group 1A\r\n,DN90XSH39C\r\n,1231 Glenarm PL.\r\n,39.740519,-104.994896,"Type 3A, Combo Pole w/Cantenna",...,CompTek,1002661,30.0,Federal Green (RAL 6012),CCI SCA-DR1-65F6-E-H2S-K(Panel Ant),Single Carrier,,NaN,NaN,9
4,Mobilitie,Mobilitie,NaN,NaN,2018 Group 1B\r\n,DN90XSH55D\r\n,1461 N. Delaware St.\r\n,39.739439,-104.993035,"Type 3A, Combo Pole w/Cantenna",...,CompTek,1002661,30.0,Federal Green (RAL 6012),CCI SCA-DR1-65F6-E-H2S-K(Panel Ant),Single Carrier,,NaN,NaN,10


In [68]:
df = (
    df.loc[:,['OWNERNAME','NODE_N','NODE_E','POLEHEIGHT_FT']]
    .dropna()
)

df.head()

,OWNERNAME,NODE_N,NODE_E,POLEHEIGHT_FT
0,Mobilitie,39.744554,-104.989407,30.0
1,Mobilitie,39.742565,-104.985513,34.0
2,Mobilitie,39.745220,-104.990277,30.0
3,Mobilitie,39.740519,-104.994896,30.0
4,Mobilitie,39.739439,-104.993035,30.0


In [69]:
import folium as fl

m = fl.Map(location=[39.742793, -105.0109598],zoom_start=11)

for lat,long,owner in zip(df.NODE_N,df.NODE_E,df.OWNERNAME):
    fl.Marker([lat,long],popup=owner).add_to(m)

m

In [70]:

from folium.plugins import HeatMap

heat = fl.Map(location=[39.742793, -105.0109598],zoo_start=11)

data = [[lat,long] for lat,long in zip(df.NODE_N,df.NODE_E)]

HeatMap(data,radius=15).add_to(heat)

heat